In [2]:
import numpy as np

In [1]:
from pdf2image import convert_from_path
pages = convert_from_path('recovery.pdf')
pages[0].save('image.jpg', 'JPEG')

In [1]:
import sys
sys.path.append(str("/remote/gpu03/schiller/skatr/models/huggingface/Phi-3_5-vision-instruct"))
from skatr_vit_phi3_v import PretrainedViT

vit_model = PretrainedViT(backbone_dir="/remote/gpu03/schiller/skatr/runs/regression_micro_ViT/2024-10-02_12-00-27")

NameError: name 'vit' is not defined

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoProcessor, AutoTokenizer, pipeline, BitsAndBytesConfig

# quantization_config = BitsAndBytesConfig(load_in_4bit=True)

if 'model' in locals():
    del model
model = AutoModelForCausalLM.from_pretrained( 
    # "/remote/gpu03/schiller/ExecLLM/models/huggingface/microsoft/Phi-3.5-MoE-instruct",
    "/remote/gpu03/schiller/skatr/models/huggingface/Phi-3_5-vision-instruct",
    device_map="cuda",  
    torch_dtype=torch.bfloat16,  
    trust_remote_code=True,
    # quantization_config=quantization_config,
    # attn_implementation="flash_attention_2",
    ignore_mismatched_sizes=True,
    output_loading_info=True,
)
    
# tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-MoE-instruct", local_files_only=True) 
# tokenizer = AutoTokenizer.from_pretrained("/remote/gpu03/schiller/ExecLLM/models/huggingface/microsoft/Phi-3.5-vision-instruct", local_files_only=True) 

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.42s/it]


In [4]:
import sys
sys.path.append("/remote/gpu03/schiller/skatr/models/huggingface/Phi-3_5-vision-instruct")
from skatr_vit_phi3_v import PretrainedViT, Phi3ImageEmbeddingSkatr

model[0].config.img_processor = {
    "image_dim_out": 144,
    "model_name": "",
    "name": "vit",
    "num_img_tokens": 144,
    "backbone_dir": "/remote/gpu03/schiller/skatr/runs/regression_micro_ViT/2024-10-02_12-00-27"
    }
embedding_config = {
    'embedding_cls': model[0].config.embd_layer['embedding_cls'],
    **model[0].config.embd_layer
    }
model[0].model.vision_embed_tokens = Phi3ImageEmbeddingSkatr(
    model[0].config,
    wte=model[0].model.embed_tokens,
    **embedding_config
    )
# model[0].model.vision_embed_tokens.img_processor = PretrainedViT(backbone_dir="/remote/gpu03/schiller/skatr/runs/regression_micro_ViT/2024-10-02_12-00-27")
# model[0].config.embd_layer['use_hd_transform'] = False
# model[0].config.embd_layer['with_learnable_separator'] = False

/remote/gpu03/schiller/skatr/models/huggingface/Phi-3_5-vision-instruct/vit_phi3_v.py:332: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state = torch.load(os.path.joi

In [5]:
model[0].save_pretrained(save_directory="/remote/gpu03/schiller/skatr/models/custom/Phi3_v_skatr", safe_serialization=False)

In [ ]:
processor = AutoProcessor.from_pretrained(
    "/remote/gpu03/schiller/skatr/models/huggingface/Phi-3_5-vision-instruct", 
    trust_remote_code=True, 
    num_crops=4,
    local_files_only=True
)

In [2]:
from PIL import Image 

images = [Image.open("image.jpg")]
placeholder = f"<|image_1|>"

messages = [
    {"role": "user", "content": placeholder+"Summarize this image."},
]

prompt = processor.tokenizer.apply_chat_template(
    messages, 
    tokenize=False, 
    add_generation_prompt=True
)

inputs = processor(prompt, images, return_tensors="pt").to("cuda:0") 

generation_args = { 
    "max_new_tokens": 1000, 
    "temperature": 0.0, 
    "do_sample": False, 
} 

generate_ids = model.generate(
    **inputs, 
    eos_token_id=processor.tokenizer.eos_token_id, 
    **generation_args
)

# remove input tokens 
generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
response = processor.batch_decode(
    generate_ids, 
    skip_special_tokens=True, 
    clean_up_tokenization_spaces=False)[0] 

print(response)

/remote/gpu03/schiller/.conda/envs/skatr/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


The image contains a graph with two y-axes and one x-axis. The left y-axis is labeled 'Network' and ranges from 0 to 10, while the right y-axis is labeled 'Net - True' and ranges from 10^-2 to 10^0. The x-axis is labeled 'Truth' and ranges from 0 to 10. There are two sets of data points: one set is plotted against the 'Network' axis and the other against the 'Net - True' axis. The data points against the 'Network' axis form a nearly straight line, indicating a linear relationship. The data points against the 'Net - True' axis form a curve that starts high on the left and decreases towards the right, with a notable dip in the middle. The graph is annotated with 'mWDM' at the top and 'MARE=8.3e-01' at the bottom, suggesting a specific metric or model used in the analysis.


In [ ]:
data_dir = '/remote/gpu02/ore/data/x2'
index = 1000
data_file = data_dir + f"/run{index}.npz"
data = np.load(data_file)
print(data['image'].shape)

In [30]:
import json

data_dict = {'image': data['image'].tolist(), 'label': data['label'].tolist()}
# data_dict = {'image': data['image'][:10,:10,550:600].tolist(), 'label': data['label'].tolist()}
data_str = json.dumps(data_dict)
# print(data_str[0:1000])

{"image": [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [31]:
messages = [
    {"role": "user", "content": data_str},
]

prompt = processor.tokenizer.apply_chat_template(
  messages, 
  tokenize=True,
  add_generation_prompt=True
)
print(len(prompt))

72728172
